In [40]:
import pandas as pd
import numpy as np

In [41]:
data1 = pd.read_csv('data/round1/prices_round_1_day_-2.csv', header = 0, sep=";")
data2 = pd.read_csv('data/round1/prices_round_1_day_-1.csv', header = 0, sep=";")
data3 = pd.read_csv('data/round1/prices_round_1_day_0.csv', header = 0, sep=";")

In [58]:
#print(data1.head())
#print(data3.tail())

#creating midprice differences matrix

data = pd.concat([data1,data2, data3])

midprices_column = data.loc[:, "mid_price"]
starfruit_midprices = midprices_column[data["product"] == "STARFRUIT"]
#print(starfruit_midprices.head())
#print(starfruit_midprices.tail())

delta_starfruit_midprices = starfruit_midprices.diff()
#print(delta_starfruit_midprices.head())

delta_starfruit_midprices_np = delta_starfruit_midprices.to_numpy()
#print(np.mean(np.abs(delta_starfruit_midprices_np[1:])))

delta_matrix = delta_starfruit_midprices_np.reshape(-1,4)
delta_matrix = delta_matrix[1:] #remove first row as it contains NaN
X = delta_matrix[:-1]
#print(X[-25:,:])

y1 = delta_matrix[:,0]
y = y1[1:]
#print(y[-5:])

#print(y.shape)
#print(X.shape)


"""ones_column = np.ones((delta_matrix.shape[0],1))
X = np.hstack((ones_column, delta_matrix))
print(X[:5, :])
print(X.shape)""" #don't need the ones column as beta constant term should be zero




1.222640754691823
[[ 0.5  0.   0.   0. ]
 [-1.   1.   0.5 -0.5]
 [ 1.5 -1.5  1.5  1. ]
 [ 2.5 -3.   4.  -3. ]
 [ 2.  -2.   0.   0. ]
 [ 0.5  0.5  0.  -1. ]
 [ 1.  -0.5  0.5 -0.5]
 [ 0.  -0.5  1.5 -2.5]
 [-0.5 -1.5  3.  -2.5]
 [ 0.5 -1.   1.  -2. ]
 [ 2.   0.5  1.  -1.5]
 [ 0.5  2.5 -2.5  0. ]
 [ 1.   0.   0.  -2.5]
 [ 3.5  0.  -3.5  2. ]
 [ 1.  -0.5  0.   1. ]
 [ 2.   0.  -2.5  0.5]
 [-1.5  1.   0.  -1. ]
 [ 1.5 -0.5  0.5  0. ]
 [-0.5 -1.   2.5 -0.5]
 [-3.  -0.5 -1.5  0. ]
 [ 0.  -1.   1.   0.5]
 [-2.5  3.   0.   0. ]
 [ 0.   0.5  0.5 -1. ]
 [ 1.  -0.5  0.  -1. ]
 [-0.5  0.5 -0.5  1. ]]
[-2.5  0.   1.  -0.5 -1.5]
(7498,)
(7498, 4)


'ones_column = np.ones((delta_matrix.shape[0],1))\nX = np.hstack((ones_column, delta_matrix))\nprint(X[:5, :])\nprint(X.shape)'

In [80]:
#doing linear regression on the matrix

def LinearRegression(data,k):

    midprices_column = data.loc[:, "mid_price"]
    starfruit_midprices = midprices_column[data["product"] == "STARFRUIT"]

    delta_starfruit_midprices = starfruit_midprices.diff()

    delta_starfruit_midprices_np = delta_starfruit_midprices.to_numpy()

    delta_matrix = delta_starfruit_midprices_np.reshape(-1,k)
    delta_matrix = delta_matrix[1:] #remove first row as it contains NaN
    X = delta_matrix[:-1]

    ones_column = np.ones((X.shape[0], 1))
    X = np.hstack((ones_column, X))

    
    y1 = delta_matrix[:,0] #price at next timestep
    y = y1[1:]

    beta = np.linalg.lstsq(X, y,rcond=None)[0]

    fit = X.dot(beta)
  
    residuals = y - fit

    SSE = np.linalg.norm(y - fit)**2
    MSE = SSE/(np.size(y))

    return [beta, fit, residuals, MSE]

print(LinearRegression(data,12)[0])
print(LinearRegression(data,12)[3])

print(LinearRegression(data,5)[0])
print(LinearRegression(data,5)[3])



[-0.03625716 -0.03958652 -0.06110437 -0.07998108 -0.11059139 -0.11079702
 -0.10927272 -0.1455748  -0.21055018 -0.33562075 -0.50009132 -0.71531163]
1.8602514703518869
[-0.07658372 -0.18116872 -0.33101596 -0.50232555 -0.72336049]
1.8628090110890168
